# Settings

In [ ]:
#Download the necessary files for the experiment

!pip install googledrivedownloader

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1Jr8BrAhWlRvXO8CcBa1XmVIgAuUdCEUx',
                                    dest_path='./content',
                                    unzip=True)

Unzipping...Done.


# Data

In [23]:
%cd res_no_temp
resdata = open('resultada.txt', encoding = 'utf-8')
resdata = resdata.read().split("\n")

[WinError 2] The system cannot find the file specified: 'res_no_temp'
C:\Users\jaimovitchg\Desktop\code\GPT-3 Experiments\notebooks


In [24]:
count = len(resdata)
i = 0
while i < count:
  if "Output://" in resdata[i]:
    resdata[i] = resdata[i][19:]
    resdata[i] = resdata[i].lstrip(" ")
    i = i + 1
  else:
    resdata.pop(i)
    count = len(resdata)

In [25]:
#Store information in a more comprehensive variable finaldata

numConcepts = 8
numBatches = 3
numTests = 5
numReps = 40


finaldata = [[]]*numReps

for r in range(0, numReps):
  finaldata[r] = [[]]*numConcepts
  for c in range(0,numConcepts):
    finaldata[r][c] = [[]]*numBatches
    for b in range(0,numBatches):
      finaldata[r][c][b] = [[]]*numTests
      for t in range(0, numTests):
        finaldata[r][c][b][t] = resdata[c*numBatches*numTests*numReps + b*numTests*numReps + t*numReps + r]

In [26]:
#Evaluation

import random as rn

seed = 23

#https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/ addapted

#Return the most repeated element in a list or a list with the most repeated elements
def most_frequent(passed_list): 
  counter = 0 
  for i in passed_list: 
    curr_frequency = passed_list.count(i) 
    if curr_frequency > counter: 
      counter = curr_frequency
      num = i 
    elif curr_frequency == counter:
      if isinstance(num, list):
        if not i in num:
          num.append(i)
      elif i != num:
        num = [num,i]
  
  return num 


aggregAnswers = [[]]*numConcepts

for c in range(0, numConcepts):
  aggregAnswers[c] = [[]]*numBatches
  for b in range(0,numBatches):
    aggregAnswers[c][b] = [""]*numTests
    for t in range(0,numTests):
      checklist = list('')
      for r in range(0,numReps):
        if finaldata[r][c][b][t] != '':
          checklist.append(finaldata[r][c][b][t])
      aggregAnswers[c][b][t] = most_frequent(checklist)
      if isinstance(aggregAnswers[c][b][t], list):
         rn.seed(seed)
         aggregAnswers[c][b][t] = rn.choice(aggregAnswers[c][b][t])


# Evaluation

## GPT-3A-T1

In [27]:
#Evaluation

correctAnswers = [['0','1','0','1','0'],['1','1','0','0','-'],['1','-','1','-','1'], ['011', '011', '011', '1', '1'], ['0', '1', '-', '1', '-'], ['00', '-', '-', '-', '-'], ['1', '0', '0', '-', '-'], ['-', '-', '1', '1', '-']]

accuracies = [[]]*numConcepts
accuracies_des = [[]]*numReps

binary_results = [[]]*numConcepts
binary_results_des = [[]]*numReps

#Calculate the accuracies
for c in range(0, numConcepts):
    accuracies[c] = [0]*numBatches
    binary_results[c] = [[]]*numBatches
    for b in range(0, numBatches):
        binary_results[c][b] = [[]]*numTests
        for t in range(0, numTests):
          if aggregAnswers[c][b][t] == correctAnswers[c][t]:
            accuracies[c][b] = accuracies[c][b] + (1/numTests)
            binary_results[c][b][t] = 1
          else:
            binary_results[c][b][t] = 0

        print("GPT3 Accuracy for C" + str(c+1) + " in Batch " + str(b+1) + ": " + str(accuracies[c][b]*100))
        

for r in range(0, numReps):
  binary_results_des[r] = [[]]*numConcepts
  accuracies_des[r] = [[]]*numConcepts
  for c in range(0, numConcepts):
    accuracies_des[r][c] = [0]*numBatches
    binary_results_des[r][c] = [[]]*numBatches
    for b in range(0, numBatches):
      binary_results_des[r][c][b] = ['']*numTests
      for t in range(0, numTests):
          if finaldata[r][c][b][t] == correctAnswers[c][t]:
            accuracies_des[r][c][b] = accuracies_des[r][c][b] + (1/numTests)
            binary_results_des[r][c][b][t] = 1
          else:
            binary_results_des[r][c][b][t] = 0


GPT3 Accuracy for C1 in Batch 1: 60.00000000000001
GPT3 Accuracy for C1 in Batch 2: 40.0
GPT3 Accuracy for C1 in Batch 3: 80.0
GPT3 Accuracy for C2 in Batch 1: 40.0
GPT3 Accuracy for C2 in Batch 2: 60.00000000000001
GPT3 Accuracy for C2 in Batch 3: 40.0
GPT3 Accuracy for C3 in Batch 1: 40.0
GPT3 Accuracy for C3 in Batch 2: 20.0
GPT3 Accuracy for C3 in Batch 3: 20.0
GPT3 Accuracy for C4 in Batch 1: 60.00000000000001
GPT3 Accuracy for C4 in Batch 2: 40.0
GPT3 Accuracy for C4 in Batch 3: 80.0
GPT3 Accuracy for C5 in Batch 1: 40.0
GPT3 Accuracy for C5 in Batch 2: 40.0
GPT3 Accuracy for C5 in Batch 3: 20.0
GPT3 Accuracy for C6 in Batch 1: 60.00000000000001
GPT3 Accuracy for C6 in Batch 2: 60.00000000000001
GPT3 Accuracy for C6 in Batch 3: 80.0
GPT3 Accuracy for C7 in Batch 1: 40.0
GPT3 Accuracy for C7 in Batch 2: 60.00000000000001
GPT3 Accuracy for C7 in Batch 3: 60.00000000000001
GPT3 Accuracy for C8 in Batch 1: 60.00000000000001
GPT3 Accuracy for C8 in Batch 2: 60.00000000000001
GPT3 Accu

In [28]:
#Additional code for the case when examples with the empty string as output are not allowed

import copy

finaldata_emp = copy.deepcopy(finaldata)
correctAnswers_emp = copy.deepcopy(correctAnswers)
aggregAnswers_emp = copy.deepcopy(aggregAnswers)

remove_pos = [[],[4],[1,3],[],[2,4],[1,2,3,4],[3,4],[0,1,4]]


for i in range(0, len(remove_pos)):
    remove_pos[i].reverse()
    for j in range(0,len(remove_pos[i])):
        del correctAnswers_emp[i][remove_pos[i][j]]
        del aggregAnswers_emp[i][b][remove_pos[i][j]]
        for b in range(0, numBatches):
            for r in range(0, numReps):
                del finaldata_emp[r][i][b][remove_pos[i][j]]

accuracies_emp = [[]]*numConcepts
accuracies_emp_des = [[]]*numReps

binary_results_emp = [[]]*numConcepts
binary_results_emp_des = [[]]*numReps

#Calculate the accuracies
for c in range(0, numConcepts):
    accuracies_emp[c] = [0]*numBatches
    binary_results_emp[c] = [[]]*numBatches
    for b in range(0, numBatches):
        binary_results_emp[c][b] = [[]]*len(correctAnswers_emp[c])
        for t in range(0, len(correctAnswers_emp[c])):
          if aggregAnswers_emp[c][b][t] == correctAnswers_emp[c][t]:
            accuracies_emp[c][b] = accuracies_emp[c][b] + (1/len(correctAnswers_emp[c]))
            binary_results_emp[c][b][t] = 1
          else:
            binary_results_emp[c][b][t] = 0

        print("GPT3 Accuracy for C" + str(c+1) + " in Batch " + str(b+1) + ": " + str(accuracies_emp[c][b]*100))
        
for r in range(0, numReps):
  binary_results_emp_des[r] = [[]]*numConcepts
  accuracies_emp_des[r] = [[]]*numConcepts
  for c in range(0, numConcepts):
    accuracies_emp_des[r][c] = [0]*numBatches
    binary_results_emp_des[r][c] = [[]]*numBatches
    for b in range(0, numBatches):
      binary_results_emp_des[r][c][b] = ['']*len(correctAnswers_emp[c])
      for t in range(0, len(correctAnswers_emp[c])):
          if finaldata_emp[r][c][b][t] == correctAnswers_emp[c][t]:
            accuracies_emp_des[r][c][b] = accuracies_emp_des[r][c][b] + (1/len(correctAnswers_emp[c]))
            binary_results_emp_des[r][c][b][t] = 1
          else:
            binary_results_emp_des[r][c][b][t] = 0

GPT3 Accuracy for C1 in Batch 1: 60.00000000000001
GPT3 Accuracy for C1 in Batch 2: 40.0
GPT3 Accuracy for C1 in Batch 3: 80.0
GPT3 Accuracy for C2 in Batch 1: 50.0
GPT3 Accuracy for C2 in Batch 2: 75.0
GPT3 Accuracy for C2 in Batch 3: 50.0
GPT3 Accuracy for C3 in Batch 1: 0
GPT3 Accuracy for C3 in Batch 2: 0
GPT3 Accuracy for C3 in Batch 3: 33.33333333333333
GPT3 Accuracy for C4 in Batch 1: 60.00000000000001
GPT3 Accuracy for C4 in Batch 2: 40.0
GPT3 Accuracy for C4 in Batch 3: 80.0
GPT3 Accuracy for C5 in Batch 1: 0
GPT3 Accuracy for C5 in Batch 2: 0
GPT3 Accuracy for C5 in Batch 3: 33.33333333333333
GPT3 Accuracy for C6 in Batch 1: 0
GPT3 Accuracy for C6 in Batch 2: 0
GPT3 Accuracy for C6 in Batch 3: 0
GPT3 Accuracy for C7 in Batch 1: 66.66666666666666
GPT3 Accuracy for C7 in Batch 2: 100.0
GPT3 Accuracy for C7 in Batch 3: 100.0
GPT3 Accuracy for C8 in Batch 1: 0
GPT3 Accuracy for C8 in Batch 2: 0
GPT3 Accuracy for C8 in Batch 3: 0
